In [ ]:
from pathlib import Path
import librosa
import matplotlib.pyplot as plt
import numpy as np
from mixes.unmixdb import UnmixDB
import seaborn as sns
from tqdm import tqdm
import pickle
sns.set_theme('paper')

In [ ]:
UNMIXDB_PATH = Path("/home/etiandre/stage/datasets/unmixdb/")
dataset = UnmixDB(UNMIXDB_PATH) 
# dataset = SyntheticDB()


In [ ]:
COLOR_CYCLE = ["blue", "green", "red", "cyan", "magenta", "yellow", "black"]
HOP = 4096
FS = 22050
results = {}
for mix in tqdm(dataset.mixes):
    tau = np.arange(0, mix.duration * FS, HOP) / FS
    gain = 20*np.log10(mix.gain(tau))
    warp = mix.warp(tau)
    plt.figure(figsize=(6,3))
    bad_duration = 0
    for i, track in enumerate(mix.tracks):
        rms = 20 * np.log10(librosa.feature.rms(y=track.audio, hop_length=HOP, frame_length=4*HOP)).flatten()
        t = librosa.times_like(rms, sr=FS, hop_length=HOP) / mix.meta[i]["speed"] + mix.meta[i]["start"]
        
        rms = np.interp(tau, t, rms, left=-200, right=-200)
        
        plt.plot(tau, rms, label=f"rms {i} {track.name}", color=COLOR_CYCLE[i])
        plt.plot(tau, gain[:,i], "--", label=f"gain {i}", color=COLOR_CYCLE[i])

        bad_zones = (rms < -60) & (gain[:,i].flatten() > -60)
        plt.fill_between(tau, -100, 0, where=bad_zones, facecolor='red', alpha=0.5)
        bad_duration += np.sum(np.diff(tau)[bad_zones[:-1]])
        
    results[mix.name] = bad_duration

    plt.ylim(-90, -5)
    plt.ylabel("dB")
    plt.xlabel("time (s)")
    # plt.legend()
    plt.title(f"{mix.name}: {bad_duration:.2f} s.")
    plt.savefig(f"badzone-{mix.name}.png")
    # plt.show()
    plt.close()
with open("badzone.pickle", "wb") as f:
    pickle.dump(results, f)

In [ ]:
import pandas as pd
with open("badzone.pickle", "rb") as f:
    results = pickle.load(f)
df = pd.DataFrame(pd.Series(results))
df

In [ ]:
import seaborn as sns
sns.set_theme('notebook')
sns.boxplot(df)